# Abstractive

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/abstractive-summarization](https://github.com/huseinzol05/Malaya/tree/master/example/abstractive-summarization).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

<div class="alert alert-warning">

This module trained heavily on news structure.
    
</div>

In [1]:
%%time
import malaya
from pprint import pprint

CPU times: user 5.46 s, sys: 1.06 s, total: 6.53 s
Wall time: 7.25 s


### List available T5 models

In [3]:
malaya.summarization.abstractive.available_t5()

INFO:root:tested on 5k CNN test set.


,Size (MB),Uncompressed Size (MB),Optimized Size (MB),ROUGE-1,ROUGE-2,ROUGE-L
small,122.0,355.6,244.0,0.33854,0.14588,0.23528
base,448.0,1300.0,895.0,0.34103,0.14994,0.23655


### Load T5

T5 is a transformer model that capable to generate abstractive summarization. In this example, we are going to use `base` model, feel free to use `small` if you find `base` is too slow.

```python
def t5(
    model: str = 'base',
    compressed: bool = True,
    optimized: bool = False,
    **kwargs,
):

    """
    Load T5 model to generate a summary given a string.

    Parameters
    ----------
    model : str, optional (default='base')
        Model architecture supported. Allowed values:

        * ``'base'`` - T5 BASE parameters.
        * ``'small'`` - T5 SMALL parameters.

    compressed: bool, optional (default=True)
        Load compressed model, but this not able to utilize malaya-gpu function. 
        This only compressed model size, but when loaded into VRAM / RAM, size uncompressed and compressed are the same.
        We prefer un-compressed model due to compressed model prone to error.
    
    optimized : bool, optional (default=False)
        if True, will load optimized uncompressed model, remove unnecessary nodes and fold batch norm to reduce model size.
        Optimized model not necessary faster, totally depends on the machine. 
        We have no concrete proof optimized model maintain same accuracy as uncompressed model.

    Returns
    -------
    result: malaya.model.t5.SUMMARIZATION class
    """
```

**For malaya-gpu user, compressed t5 very fragile and we suggest use `compressed=False`. Uncompressed model also can utilise GPU usage more efficient**.

In [3]:
model = malaya.summarization.abstractive.t5(model = 'base')


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /Users/huseinzolkepli/Malaya/summarize-argmax/t5/base/model/variables/variables


#### Predict

For T5, we only provided `greedy_decoder` method to predict.

#### summarization mode

T5 in Malaya provided 2 different modes for summarization,

1. generate summary,

```python
model.greedy_decoder(string, mode = 'ringkasan')
```

2. generate title,

```python
model.greedy_decoder(string, mode = 'tajuk')
```

default is `ringkasan`,

```python
def greedy_decoder(
    self,
    string: str,
    mode: str = 'ringkasan',
    postprocess: bool = True,
    **kwargs,
):
    """
    Summarize a string. Decoder is greedy decoder with beam width size 1, alpha 0.5 .

    Parameters
    ----------
    string: str
    mode: str
        mode for summarization. Allowed values:

        * ``'ringkasan'`` - summarization for long sentence, eg, news summarization.
        * ``'tajuk'`` - title summarization for long sentence, eg, news title.
    postprocess: bool, optional (default=True)
        If True, will filter sentence generated using ROUGE score and removed international news publisher.

    Returns
    -------
    result: str
    """
```

I am going to simply copy paste some local news into this notebook. I will search about `isu mahathir` in google news, [link here](https://www.google.com/search?q=isu+mahathir&sxsrf=ALeKk02V_bAJC3sSrV38JQgGYWL_mE0biw:1589951900053&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjapNmx2MHpAhVp_XMBHRt7BEQQ_AUoAnoECCcQBA&biw=1440&bih=648&dpr=2).

**link**: https://www.hmetro.com.my/mutakhir/2020/05/580438/peletakan-jawatan-tun-m-ditolak-bukan-lagi-isu

**Title**: Peletakan jawatan Tun M ditolak, bukan lagi isu.

**Body**: PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu.

Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.

Bekas Setiausaha Agung Bersatu Datuk Marzuki Yahya berkata, pada mesyuarat itu MPT sebulat suara menolak peletakan jawatan Dr Mahathir.

"Jadi ini agak berlawanan dengan keputusan yang kita sudah buat. Saya tak faham bagaimana Jabatan Pendaftar Pertubuhan Malaysia (JPPM) kata peletakan jawatan itu sah sedangkan kita sudah buat keputusan di dalam mesyuarat, bukan seorang dua yang buat keputusan.

"Semua keputusan mesti dibuat melalui parti. Walau apa juga perbincangan dibuat di luar daripada keputusan mesyuarat, ini bukan keputusan parti.

"Apa locus standy yang ada pada Setiausaha Kerja untuk membawa perkara ini kepada JPPM. Seharusnya ia dibawa kepada Setiausaha Agung sebagai pentadbir kepada parti," katanya kepada Harian Metro.

Beliau mengulas laporan media tempatan hari ini mengenai pengesahan JPPM bahawa Dr Mahathir tidak lagi menjadi Pengerusi Bersatu berikutan peletakan jawatannya di tengah-tengah pergolakan politik pada akhir Februari adalah sah.

Laporan itu juga menyatakan, kedudukan Muhyiddin Yassin memangku jawatan itu juga sah.

Menurutnya, memang betul Dr Mahathir menghantar surat peletakan jawatan, tetapi ditolak oleh MPT.

"Fasal yang disebut itu terpakai sekiranya berhenti atau diberhentikan, tetapi ini mesyuarat sudah menolak," katanya.

Marzuki turut mempersoal kenyataan media yang dibuat beberapa pimpinan parti itu hari ini yang menyatakan sokongan kepada Perikatan Nasional.

"Kenyataan media bukanlah keputusan rasmi. Walaupun kita buat 1,000 kenyataan sekali pun ia tetap tidak merubah keputusan yang sudah dibuat di dalam mesyuarat. Kita catat di dalam minit apa yang berlaku di dalam mesyuarat," katanya.

In [6]:
string = """
PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu.

Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.

Bekas Setiausaha Agung Bersatu Datuk Marzuki Yahya berkata, pada mesyuarat itu MPT sebulat suara menolak peletakan jawatan Dr Mahathir.

"Jadi ini agak berlawanan dengan keputusan yang kita sudah buat. Saya tak faham bagaimana Jabatan Pendaftar Pertubuhan Malaysia (JPPM) kata peletakan jawatan itu sah sedangkan kita sudah buat keputusan di dalam mesyuarat, bukan seorang dua yang buat keputusan.

"Semua keputusan mesti dibuat melalui parti. Walau apa juga perbincangan dibuat di luar daripada keputusan mesyuarat, ini bukan keputusan parti.

"Apa locus standy yang ada pada Setiausaha Kerja untuk membawa perkara ini kepada JPPM. Seharusnya ia dibawa kepada Setiausaha Agung sebagai pentadbir kepada parti," katanya kepada Harian Metro.

Beliau mengulas laporan media tempatan hari ini mengenai pengesahan JPPM bahawa Dr Mahathir tidak lagi menjadi Pengerusi Bersatu berikutan peletakan jawatannya di tengah-tengah pergolakan politik pada akhir Februari adalah sah.

Laporan itu juga menyatakan, kedudukan Muhyiddin Yassin memangku jawatan itu juga sah.

Menurutnya, memang betul Dr Mahathir menghantar surat peletakan jawatan, tetapi ditolak oleh MPT.

"Fasal yang disebut itu terpakai sekiranya berhenti atau diberhentikan, tetapi ini mesyuarat sudah menolak," katanya.

Marzuki turut mempersoal kenyataan media yang dibuat beberapa pimpinan parti itu hari ini yang menyatakan sokongan kepada Perikatan Nasional.

"Kenyataan media bukanlah keputusan rasmi. Walaupun kita buat 1,000 kenyataan sekali pun ia tetap tidak merubah keputusan yang sudah dibuat di dalam mesyuarat. Kita catat di dalam minit apa yang berlaku di dalam mesyuarat," katanya.
"""

In [4]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [8]:
string = cleaning(string)

#### generate ringkasan

In [6]:
pprint(model.greedy_decoder(string, mode = 'ringkasan'))

('Kenyataan media yang dibuat oleh kepimpinan parti adalah sah. Tidak ada '
 'persoalan peletakan jawatan Dr Mahathir adalah sah atau tidak. Ia sudah '
 'diputuskan oleh semua pihak termasuk Presiden, Tan Sri Muhyiddin Yassin')


#### generate tajuk

In [9]:
model.greedy_decoder(string, mode = 'tajuk')

'Bukan pertikai keputusan MPT - Marzuki'

**Link**: https://www.malaysiakini.com/news/525953

**Title**: Mahathir jangan hipokrit isu kes mahkamah Riza, kata Takiyuddin

**Body**: Menteri undang-undang Takiyuddin Hassan berkata kerajaan berharap Dr Mahathir Mohamad tidak bersikap hipokrit dengan mengatakan beliau tertanya-tanya dan tidak faham dengan keputusan mahkamah melepas tanpa membebaskan (DNAA) Riza Aziz, anak tiri bekas perdana menteri Najib Razak, dalam kes pengubahan wang haram membabitkan dana 1MDB.

Pemimpin PAS itu berkata ini kerana keputusan itu dibuat oleh peguam negara dan dilaksanakan oleh timbalan pendakwa raya yang mengendalikan kes tersebut pada akhir 2019.

“Saya merujuk kepada kenyataan Dr Mahathir tentang tindakan Mahkamah Sesyen memberikan pelepasan tanpa pembebasan (discharge not amounting to acquittal) kepada Riza Aziz baru-baru ini.

“Kerajaan berharap Dr Mahathir tidak bersikap hipokrit dengan mengatakan beliau ‘tertanya-tanya’, keliru dan tidak faham terhadap suatu keputusan yang dibuat oleh Peguam Negara dan dilaksanakan oleh Timbalan Pendakwa Raya yang mengendalikan kes ini pada akhir tahun 2019,” katanya dalam satu kenyataan hari ini.

Riza pada Khamis dilepas tanpa dibebaskan daripada lima pertuduhan pengubahan wang berjumlah AS$248 juta (RM1.08 bilion).

Dalam persetujuan yang dicapai antara pihak Riza dan pendakwaan, beliau dilepas tanpa dibebaskan atas pertuduhan itu dengan syarat memulangkan semula aset dari luar negara dengan nilai anggaran AS$107.3 juta (RM465.3 juta).

Ekoran itu, Mahathir antara lain menyuarakan kekhuatirannya berkenaan persetujuan itu dan mempersoalkan jika pihak yang didakwa atas tuduhan mencuri boleh terlepas daripada tindakan jika memulangkan semula apa yang dicurinya.

"Dia curi berbilion-bilion...Dia bagi balik kepada kerajaan. Dia kata kepada kerajaan, 'Nah, duit yang aku curi. Sekarang ini, jangan ambil tindakan terhadap aku.' Kita pun kata, 'Sudah bagi balik duit okey lah'," katanya.

Menjelaskan bahawa beliau tidak mempersoalkan keputusan mahkamah, Mahathir pada masa sama berkata ia menunjukkan undang-undang mungkin perlu dipinda.

Mengulas lanjut, Takiyuddin yang juga setiausaha agung PAS berkata
kenyataan Mahathir tidak munasabah sebagai bekas perdana menteri.

"Kerajaan berharap Dr Mahathir tidak terus bertindak mengelirukan rakyat dengan mengatakan beliau ‘keliru’.

“Kerajaan PN akan terus bertindak mengikut undang-undang dan berpegang kepada prinsip kebebasan badan kehakiman dan proses perundangan yang sah,” katanya.

In [9]:
string = """
Menteri undang-undang Takiyuddin Hassan berkata kerajaan berharap Dr Mahathir Mohamad tidak bersikap hipokrit dengan mengatakan beliau tertanya-tanya dan tidak faham dengan keputusan mahkamah melepas tanpa membebaskan (DNAA) Riza Aziz, anak tiri bekas perdana menteri Najib Razak, dalam kes pengubahan wang haram membabitkan dana 1MDB.

Pemimpin PAS itu berkata ini kerana keputusan itu dibuat oleh peguam negara dan dilaksanakan oleh timbalan pendakwa raya yang mengendalikan kes tersebut pada akhir 2019.

“Saya merujuk kepada kenyataan Dr Mahathir tentang tindakan Mahkamah Sesyen memberikan pelepasan tanpa pembebasan (discharge not amounting to acquittal) kepada Riza Aziz baru-baru ini.

“Kerajaan berharap Dr Mahathir tidak bersikap hipokrit dengan mengatakan beliau ‘tertanya-tanya’, keliru dan tidak faham terhadap suatu keputusan yang dibuat oleh Peguam Negara dan dilaksanakan oleh Timbalan Pendakwa Raya yang mengendalikan kes ini pada akhir tahun 2019,” katanya dalam satu kenyataan hari ini.

Riza pada Khamis dilepas tanpa dibebaskan daripada lima pertuduhan pengubahan wang berjumlah AS$248 juta (RM1.08 bilion).

Dalam persetujuan yang dicapai antara pihak Riza dan pendakwaan, beliau dilepas tanpa dibebaskan atas pertuduhan itu dengan syarat memulangkan semula aset dari luar negara dengan nilai anggaran AS$107.3 juta (RM465.3 juta).

Ekoran itu, Mahathir antara lain menyuarakan kekhuatirannya berkenaan persetujuan itu dan mempersoalkan jika pihak yang didakwa atas tuduhan mencuri boleh terlepas daripada tindakan jika memulangkan semula apa yang dicurinya.

"Dia curi berbilion-bilion...Dia bagi balik kepada kerajaan. Dia kata kepada kerajaan, 'Nah, duit yang aku curi. Sekarang ini, jangan ambil tindakan terhadap aku.' Kita pun kata, 'Sudah bagi balik duit okey lah'," katanya.

Menjelaskan bahawa beliau tidak mempersoalkan keputusan mahkamah, Mahathir pada masa sama berkata ia menunjukkan undang-undang mungkin perlu dipinda.

Mengulas lanjut, Takiyuddin yang juga setiausaha agung PAS berkata
kenyataan Mahathir tidak munasabah sebagai bekas perdana menteri.

"Kerajaan berharap Dr Mahathir tidak terus bertindak mengelirukan rakyat dengan mengatakan beliau ‘keliru’.

“Kerajaan PN akan terus bertindak mengikut undang-undang dan berpegang kepada prinsip kebebasan badan kehakiman dan proses perundangan yang sah,” katanya.
"""

string = cleaning(string)

In [10]:
pprint(model.summarize(string, mode = 'ringkasan'))

('"Kerajaan berharap Dr Mahathir tidak hipokrit," kata menteri undang-undang. '
 'Riza Aziz, anak tiri Najib Razak, dilepas tanpa dibebaskan atas tuduhan '
 'pengubahan wang haram. Mahathir mengatakan dia mempersoalkan jika pihak yang '
 'didakwa mencuri boleh terlepas tindakan')


In [15]:
model.decoder(string, mode = 'tajuk')

'Kerajaan harap Dr M tak hipokrit'

### List available LM Transformer models

Problem with T5 models, it built on top of mesh-tensorflow, so the input must size of 1. So we use Tensor2Tensor library to train exact model as T5 with dynamic size of batch.

In [2]:
malaya.summarization.abstractive.available_transformer()

INFO:root:tested on 5k CNN test set.


,Size (MB),Quantized Size (MB),ROUGE-1,ROUGE-2,ROUGE-L
small,379.0,120.0,0.33000,0.13417,0.23059
base,832.0,279.0,0.33209,0.13622,0.23348


### Load Transformer

To load 8-bit quantized model, simply pass `quantized = True`, default is `False`.

We can expect slightly accuracy drop from quantized model, and not necessary faster than normal 32-bit float model, totally depends on machine.

In [3]:
model = malaya.summarization.abstractive.transformer(model = 'small')
quantized_model = malaya.summarization.abstractive.transformer(model = 'small', quantized = True)

#### summarization mode

T5 in Malaya provided 2 different modes for summarization,

1. generate summary,

```python
model.summarize(string, mode = 'ringkasan')
```

2. generate title,

```python
model.summarize(string, mode = 'tajuk')
```

default is `ringkasan`.

In [5]:
string = """
KUALA LUMPUR: Hakim Mahkamah Tinggi, Mohd Nazlan Mohd Ghazali menyifatkan kes penyelewengan dana RM42 juta milik SRC International Sdn Bhd dihadapi Datuk Seri Najib Razak adalah kesalahan salah guna kedudukan, pecah amanah jenayah dan pengubahan wang haram yang paling teruk.

Mohd Nazlan yang mensabitkan Najib terhadap kesemua tujuh tuduhan dan memerintahkan bekas Perdana Menteri itu dipenjara 12 tahun, dan didenda RM210 juta, berkata ia bukan sahaja disebabkan oleh alasan bagaimana jenayah itu dilakukan, malah kes berprofil tinggi berkenaan turut membabitkan sejumlah wang yang sangat besar.

Melalui alasan penghakiman penuh setebal 801 muka surat itu, Mohd Nazlan, berkata kes terbabit mempunyai elemen yang memberikan kesan ke atas kepentingan awam kerana dana RM42 juta itu adalah milik Kementerian Kewangan (Diperbadankan) (MKD) yang berkemungkinan berasal daripada dana pencen Kumpulan Wang Persaraan (Diperbadankan) (KWAP) berjumlah RM4 bilion.

"Dan yang paling penting ia membabitkan individu yang pada ketika itu berada dalam pada tertinggi dalam kerajaan," katanya.

Pada 28 Julai lalu, Mohd Nazlan memerintahkan Najib dipenjarakan 10 tahun masing-masing bagi tiga tuduhan pecah amanah wang RM42 juta milik SRC.

Hakim turut memerintahkan Najib dipenjara 12 tahun dan denda RM210 juta (jika gagal bayar, lima tahun penjara) bagi tuduhan menyalahgunakan kedudukan.

Bagi tuduhan pengubahan wang haram pula, Mohd Nazlan memerintahkan Najib dipenjara 10 tahun bagi setiap tuduhan.

Sementara itu, Mohd Nazlan berkata, Najib selaku tertuduh tidak menunjukkan penyesalan, malah mempertahankan pembelaan beliau tidak mengetahui mengenai wang RM42 juta milik SRC itu dalam rayuannya bagi diringankan hukuman.

"Tetapi saya tidak boleh menafikan beliau adalah Perdana Menteri negara ini dan tidak boleh mempersoalkan sumbangannya untuk kebaikan dan kesejahteraan masyarakat dalam pelbagai cara kerana beliau adalah Perdana Menteri selama sembilan tahun.

"Sejarah politik akan terus diperdebatkan sama ada dari segi keseimbangan, beliau melakukan lebih banyak kebaikan daripada keburukan.

"Walau apa pun, ia adalah tidak selari dengan idea sesebuah pentadbiran negara yang bersih daripada rasuah yang tidak boleh bertolak ansur dengan sebarang penyalahgunaan kuasa," katanya.

Mahkamah Rayuan menetapkan pada 15 Oktober ini bagi pengurusan kes rayuan Najib terhadap sabitan dan hukuman terhadapnya.
"""

string = cleaning(string)

#### Predict using greedy decoder

```python
def greedy_decoder(
    self,
    strings: List[str],
    mode: str = 'ringkasan',
    postprocess: bool = True,
    **kwargs,
):
    """
    Summarize strings using greedy decoder.

    Parameters
    ----------
    strings: List[str]
    mode: str
        mode for summarization. Allowed values:

        * ``'ringkasan'`` - summarization for long sentence, eg, news summarization.
        * ``'tajuk'`` - title summarization for long sentence, eg, news title.
    postprocess: bool, optional (default=True)
        If True, will filter sentence generated using ROUGE score and removed international news publisher.

    Returns
    -------
    result: List[str]
    """
```

In [12]:
pprint(model.greedy_decoder([string], mode = 'tajuk'))

['Najib terlibat salah guna kedudukan, pecah amanah, pengubahan wang haram']


In [13]:
pprint(quantized_model.greedy_decoder([string], mode = 'tajuk'))

['Najib terlibat kes salah guna kedudukan, pecah amanah - hakim']


In [14]:
pprint(model.greedy_decoder([string], mode = 'ringkasan'))

['Kes Najib salah guna kedudukan, pecah amanah jenayah dan pengubahan wang '
 'haram yang paling teruk. Hakim mengatakan kes itu mempunyai elemen yang '
 'memberi kesan kepada kepentingan awam. Najib diperintahkan dipenjara 10 '
 'tahun dan denda $210 juta.']


In [15]:
pprint(quantized_model.greedy_decoder([string], mode = 'ringkasan'))

['Kes Najib salah guna kedudukan, pecah amanah jenayah dan keburukan wang '
 'haram yang paling teruk. Hakim mengatakan kes itu mempunyai elemen yang '
 'memberi kesan kepada kepentingan awam. Najib diperintahkan dipenjara 10 '
 'tahun masing-masing untuk tiga tuduhan pecah amanah wang RM42 juta.']


#### Predict using beam decoder

```python
def beam_decoder(
    self,
    strings: List[str],
    mode: str = 'ringkasan',
    postprocess: bool = True,
    **kwargs,
):
    """
    Summarize strings using beam decoder, beam width size 3, alpha 0.5 .

    Parameters
    ----------
    strings: List[str]
    mode: str
        mode for summarization. Allowed values:

        * ``'ringkasan'`` - summarization for long sentence, eg, news summarization.
        * ``'tajuk'`` - title summarization for long sentence, eg, news title.
    postprocess: bool, optional (default=True)
        If True, will filter sentence generated using ROUGE score and removed international news publisher.

    Returns
    -------
    result: List[str]
    """
```

In [16]:
pprint(model.beam_decoder([string], mode = 'tajuk'))

['Najib salah guna kedudukan, pecah amanah, pengubahan wang haram']


In [17]:
pprint(quantized_model.beam_decoder([string], mode = 'tajuk'))

['Najib salah guna kedudukan, pecah amanah, pengubahan wang haram']


In [18]:
pprint(model.beam_decoder([string], mode = 'ringkasan'))

['Kes Najib salah guna kedudukan, pecah amanah jenayah dan pengubahan wang '
 'haram yang paling teruk. Hakim memerintah Najib dipenjara 12 tahun, denda '
 'RM210 juta.']


In [19]:
pprint(quantized_model.beam_decoder([string], mode = 'ringkasan'))

['Kes Najib salah guna kedudukan, pecah amanah jenayah, pengubahan wang haram '
 'yang paling teruk. Hakim mengatakan kes itu mempunyai elemen yang memberi '
 'kesan kepada kepentingan awam. Najib diperintahkan dipenjara 10 tahun '
 'masing-masing untuk tiga tuduhan pecah amanah wang RM42 juta.']


#### Predict using nucleus decoder

```python
def nucleus_decoder(
    self,
    strings: List[str],
    mode: str = 'ringkasan',
    top_p: float = 0.7,
    postprocess: bool = True,
    **kwargs,
):
    """
    Summarize strings using nucleus sampling.

    Parameters
    ----------
    strings: List[str]
    mode: str
        mode for summarization. Allowed values:

        * ``'ringkasan'`` - summarization for long sentence, eg, news summarization.
        * ``'tajuk'`` - title summarization for long sentence, eg, news title.
    top_p: float, (default=0.7)
        cumulative distribution and cut off as soon as the CDF exceeds `top_p`.
    postprocess: bool, optional (default=True)
        If True, will filter sentence generated using ROUGE score and removed international news publisher.

    Returns
    -------
    result: List[str]
    """
```

In [20]:
pprint(model.nucleus_decoder([string], mode = 'tajuk'))

['Najib terlibat kesalahan salah guna kedudukan, pecah amanah dan pengubahan '
 'wang haram']


In [6]:
pprint(model.nucleus_decoder([string], mode = 'ringkasan'))

['Kes Najib salah guna kedudukan, pecah amanah jenayah, pengubahan wang haram '
 'yang paling teruk. Hakim: Kes melibatkan Najib antara tujuh pertuduhan dan '
 'memerintahkan bekas Perdana Menteri dipenjara 12 tahun. Najib diperintahkan '
 'pada 5 Mac untuk tiga tuduhan pecah amanah wang berjumlah RM42 juta.']
